In [3]:
from mmcv import Config
cfg = Config.fromfile('./configs/textrecog/sar/sar_r31_parallel_decoder_alpr.py')

c:\Users\Loune\Documents\GetThePlate\gtp\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from mmdet.apis import set_random_seed

# Set up working dir to save files and logs.
cfg.work_dir = './alpr_results'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.001 / 8
cfg.lr_config.warmup = None
# Choose to log training results every 200 images to reduce the size of log file. 
cfg.log_config.interval = 200

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

Config:
log_config = dict(interval=200, hooks=[dict(type='TextLoggerHook')])
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = None
resume_from = None
workflow = [('train', 1)]
opencv_num_threads = 0
mp_start_method = 'fork'
label_convertor = dict(
    type='AttnConvertor', dict_type='DICT90', with_unknown=True)
model = dict(
    type='SARNet',
    backbone=dict(type='ResNet31OCR'),
    encoder=dict(
        type='SAREncoder', enc_bi_rnn=False, enc_do_rnn=0.1, enc_gru=False),
    decoder=dict(
        type='ParallelSARDecoder',
        enc_bi_rnn=False,
        dec_bi_rnn=False,
        dec_do_rnn=0,
        dec_gru=False,
        pred_dropout=0.1,
        d_k=512,
        pred_concat=True),
    loss=dict(type='SARLoss'),
    label_convertor=dict(
        type='AttnConvertor', dict_type='DICT90', with_unknown=True),
    max_seq_len=30)
optimizer = dict(type='Adam', lr=0.000125)
optimizer_config = dict(grad_clip=None)
lr_config = dict(policy='step', step=[3, 4], warmup=N

In [8]:
cfg.optimizer.lr 

0.000125

In [21]:
from mmocr.datasets import build_dataset
from mmocr.models import build_detector
from mmocr.apis import train_detector
import mmcv
import os.path as osp

# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

c:\Users\Loune\Documents\GetThePlate\Recognition_Module\mmocr\mmocr\apis\utils.py:53: UserWarning: Remove "MultiRotateAugOCR" to support batch inference since samples_per_gpu > 1.
  warnings.warn(warning_msg)
2022-07-20 15:43:58,149 - mmocr - INFO - Start running, host: Loune@DESKTOP-IKB6UAR, work_dir: c:\Users\Loune\Documents\GetThePlate\Recognition_Module\mmocr\alpr_results
2022-07-20 15:43:58,166 - mmocr - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(NORMAL      ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) EvalHook                           
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdater

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 252/252, 1.3 task/s, elapsed: 194s, ETA:     0s

2022-07-21 21:02:38,571 - mmocr - INFO - 
Evaluating tests/data/ocr_alpr_dataset/plaques_labels_test.txt with 252 images now
2022-07-21 21:02:40,009 - mmocr - INFO - Epoch(val) [1][42]	0_char_recall: 0.9083, 0_char_precision: 0.9121, 0_word_acc: 0.7738, 0_word_acc_ignore_case: 0.7738, 0_word_acc_ignore_case_symbol: 0.7738, 0_1-N.E.D: 0.8952
2022-07-21 21:05:56,734 - mmocr - INFO - Epoch [2][200/113500]	lr: 1.250e-04, eta: 4 days, 20:49:53, time: 0.980, data_time: 0.052, memory: 1113, loss_ce: 0.2366, loss: 0.2366
2022-07-21 21:09:31,500 - mmocr - INFO - Epoch [2][400/113500]	lr: 1.250e-04, eta: 4 days, 20:48:45, time: 1.074, data_time: 0.004, memory: 1113, loss_ce: 0.2619, loss: 0.2619
2022-07-21 21:13:19,418 - mmocr - INFO - Epoch [2][600/113500]	lr: 1.250e-04, eta: 4 days, 20:48:28, time: 1.140, data_time: 0.004, memory: 1113, loss_ce: 0.2534, loss: 0.2534
2022-07-21 21:17:05,041 - mmocr - INFO - Epoch [2][800/113500]	lr: 1.250e-04, eta: 4 days, 20:48:01, time: 1.128, data_time: 0.00

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 252/252, 1.1 task/s, elapsed: 227s, ETA:     0s

2022-07-22 23:48:15,582 - mmocr - INFO - 
Evaluating tests/data/ocr_alpr_dataset/plaques_labels_test.txt with 252 images now
2022-07-22 23:48:16,152 - mmocr - INFO - Epoch(val) [2][42]	0_char_recall: 0.9285, 0_char_precision: 0.9317, 0_word_acc: 0.8175, 0_word_acc_ignore_case: 0.8175, 0_word_acc_ignore_case_symbol: 0.8175, 0_1-N.E.D: 0.9187
2022-07-22 23:50:35,686 - mmocr - INFO - Epoch [3][200/113500]	lr: 1.250e-04, eta: 3 days, 11:45:27, time: 0.695, data_time: 0.045, memory: 1113, loss_ce: 0.2420, loss: 0.2420
2022-07-22 23:52:59,613 - mmocr - INFO - Epoch [3][400/113500]	lr: 1.250e-04, eta: 3 days, 11:41:40, time: 0.719, data_time: 0.003, memory: 1113, loss_ce: 0.2459, loss: 0.2459
2022-07-22 23:55:27,052 - mmocr - INFO - Epoch [3][600/113500]	lr: 1.250e-04, eta: 3 days, 11:37:58, time: 0.737, data_time: 0.003, memory: 1113, loss_ce: 0.2439, loss: 0.2439
2022-07-22 23:57:55,260 - mmocr - INFO - Epoch [3][800/113500]	lr: 1.250e-04, eta: 3 days, 11:34:17, time: 0.741, data_time: 0.00

...

2022-07-24 00:02:13,091 - mmocr - INFO - 
Evaluating tests/data/ocr_alpr_dataset/plaques_labels_test.txt with 252 images now
2022-07-24 00:02:13,237 - mmocr - INFO - Epoch(val) [3][42]	0_char_recall: 0.9148, 0_char_precision: 0.9183, 0_word_acc: 0.7976, 0_word_acc_ignore_case: 0.7976, 0_word_acc_ignore_case_symbol: 0.7976, 0_1-N.E.D: 0.9019
2022-07-24 00:04:30,713 - mmocr - INFO - Epoch [4][200/113500]	lr: 1.250e-05, eta: 2 days, 5:17:28, time: 0.684, data_time: 0.043, memory: 1113, loss_ce: 0.2300, loss: 0.2300
2022-07-24 00:06:51,862 - mmocr - INFO - Epoch [4][400/113500]	lr: 1.250e-05, eta: 2 days, 5:14:20, time: 0.705, data_time: 0.003, memory: 1113, loss_ce: 0.2222, loss: 0.2222
2022-07-24 00:09:16,427 - mmocr - INFO - Epoch [4][600/113500]	lr: 1.250e-05, eta: 2 days, 5:11:15, time: 0.723, data_time: 0.003, memory: 1113, loss_ce: 0.2286, loss: 0.2286
2022-07-24 00:11:41,737 - mmocr - INFO - Epoch [4][800/113500]	lr: 1.250e-05, eta: 2 days, 5:08:10, time: 0.726, data_time: 0.003, m

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 252/252, 1.2 task/s, elapsed: 208s, ETA:     0s

2022-07-25 03:35:00,873 - mmocr - INFO - 
Evaluating tests/data/ocr_alpr_dataset/plaques_labels_test.txt with 252 images now
2022-07-25 03:35:01,080 - mmocr - INFO - Epoch(val) [4][42]	0_char_recall: 0.9350, 0_char_precision: 0.9371, 0_word_acc: 0.8214, 0_word_acc_ignore_case: 0.8214, 0_word_acc_ignore_case_symbol: 0.8214, 0_1-N.E.D: 0.9224
2022-07-25 03:37:29,429 - mmocr - INFO - Epoch [5][200/113500]	lr: 1.250e-06, eta: 1 day, 2:49:07, time: 0.738, data_time: 0.055, memory: 1113, loss_ce: 0.2210, loss: 0.2210
2022-07-25 03:40:06,579 - mmocr - INFO - Epoch [5][400/113500]	lr: 1.250e-06, eta: 1 day, 2:46:14, time: 0.786, data_time: 0.003, memory: 1113, loss_ce: 0.2388, loss: 0.2388
2022-07-25 03:42:45,482 - mmocr - INFO - Epoch [5][600/113500]	lr: 1.250e-06, eta: 1 day, 2:43:20, time: 0.794, data_time: 0.003, memory: 1113, loss_ce: 0.2353, loss: 0.2353
2022-07-25 03:45:29,351 - mmocr - INFO - Epoch [5][800/113500]	lr: 1.250e-06, eta: 1 day, 2:40:28, time: 0.819, data_time: 0.004, memor

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 252/252, 1.2 task/s, elapsed: 213s, ETA:     0s

2022-07-26 07:55:03,687 - mmocr - INFO - 
Evaluating tests/data/ocr_alpr_dataset/plaques_labels_test.txt with 252 images now
2022-07-26 07:55:03,883 - mmocr - INFO - Epoch(val) [5][42]	0_char_recall: 0.9338, 0_char_precision: 0.9360, 0_word_acc: 0.8175, 0_word_acc_ignore_case: 0.8175, 0_word_acc_ignore_case_symbol: 0.8175, 0_1-N.E.D: 0.9197
